# COGS 108 - Data Checkpoint

# Names

- Yiyao Liu (A16314015)
- Justin Lee (A15802198)
- Duye Liu (A16601637)
- Sally Yu (A16519059)
- Jie Mei (MJ) (A15825777)


<a id='research_question'></a>
# Research Question

*Is there a correlation between the dropout rates of universities in the United States and the number of billionaire alumni since 2000?*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- **Dataset Name:** 2021 Forbes Billionaires
- **Link to the dataset:** https://www.kaggle.com/roysouravcu/forbes-billionaires-of-2021
- **Number of observations:** 2752

This dataset contains the data on worldwide billionaires ranked by Forbes. The attributes include the billionaires' names, networth, countries, soruces, ranks, and industries. We have added columns of educations and dropouts, which include the institutions where billionaires graduated from and whether they were dropouts. These information, along with those from the college completion institution dataset, will help us to explore and address our research question. 

- **Dataset Name:** College Completion Institution Details
- **Link to the dataset:** https://data.world/databeats/college-completion
- **Number of observations:** 3798

This dataset provides data on a large number of U.S. colleges, which includes each college's location, type of institution, and most importantly graduation completion rate, which we can use to proportion of students who were unable to graduate within 4 years.

**We will combine these two datasets using the names of colleges as unique identifiers.**

# Setup

In [1]:
import requests  
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Data Cleaning

**From the 2021 Forbes Billionaires dataset we found online(https://www.kaggle.com/roysouravcu/forbes-billionaires-of-2021), we stored their names in a csv file named billionaire_raw.csv.**

**Read in billionaire_raw.csv and take a look at it**

In [2]:
df = pd.read_csv('billionaire_raw.csv')
df.head()

,Name,NetWorth,Country,Source,Rank,Age,Industry
0,Jeff Bezos,$177 B,United States,Amazon,1,57.0,Technology
1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,49.0,Automotive
2,Bernard Arnault & family,$150 B,France,LVMH,3,72.0,Fashion & Retail
3,Bill Gates,$124 B,United States,Microsoft,4,65.0,Technology
4,Mark Zuckerberg,$97 B,United States,Facebook,5,36.0,Technology


**Since our research question focuses on the correlation between the universities and billionaires in the United States, We will be filtering out information of billionaires outside of the States.**

In [3]:
df = df[df['Country'] == 'United States']
df.head()

,Name,NetWorth,Country,Source,Rank,Age,Industry
0,Jeff Bezos,$177 B,United States,Amazon,1,57.0,Technology
1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,49.0,Automotive
3,Bill Gates,$124 B,United States,Microsoft,4,65.0,Technology
4,Mark Zuckerberg,$97 B,United States,Facebook,5,36.0,Technology
5,Warren Buffett,$96 B,United States,Berkshire Hathaway,6,90.0,Finance & Investments


**Let's take a look at the dimensions of our current billionaire dataset**

In [4]:
df.shape

(724, 7)

**Let's check if there are any nans in our billionaire dataset.**

In [5]:
df.isnull().sum()

Name        0
NetWorth    0
Country     0
Source      0
Rank        0
Age         4
Industry    0
dtype: int64

**As you can see, there are four nans in the age column, some billionaires might be less inclined to share their ages. Since the quantity of nans is small, we decide to keep these rows that contain the nans. The reason is that we don't want to miss out any potential valuable information on the billionaires**

**Now we still need data on which universities these U.S. billionaires graduated or dropped out from. We will use BeautifulSoup to scrape this data from forbes worlds' billionaires list of 2021, where they list each billionaire's education background on the profile page of each billionaire (for example, https://www.forbes.com/profile/elon-musk/?sh=37e6593f7999).**

**We have 724 U.S. billionaires. For scraping, we need to generate an url for each of them using their names. Therefore, we wrote a function to standardizing their names with the corresponding urls that contain their enducation information.**

In [6]:
# define a function that changes names into urls
def standardize_names(str_in):
    
    try:
        ## change strings to lowercases
        str_in = str_in.lower()
        
        ## strip all the leading and trailing spaces 
        str_in = str_in.strip()
        
        ## replace whitespaces with dashes
        output = str_in.replace(' ', '-')
        
        ## initialize the billionaire profile url
        url_start = 'https://www.forbes.com/profile/'
        
        ## output cpntaining the url and names
        output = url_start + output
        
    except:
        
        ## avoid outputs containing nans
        output = np.nan
        
    ## return the output   
    return output

**Let's apply the standardize_names function to the Name column and store these results into a new column called urls**

In [7]:
df['urls'] = df['Name'].apply(standardize_names)

**Let's take a look at our current billionaire dataframe**

In [8]:
df.head()

,Name,NetWorth,Country,Source,Rank,Age,Industry,urls
0,Jeff Bezos,$177 B,United States,Amazon,1,57.0,Technology,https://www.forbes.com/profile/jeff-bezos
1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,49.0,Automotive,https://www.forbes.com/profile/elon-musk
3,Bill Gates,$124 B,United States,Microsoft,4,65.0,Technology,https://www.forbes.com/profile/bill-gates
4,Mark Zuckerberg,$97 B,United States,Facebook,5,36.0,Technology,https://www.forbes.com/profile/mark-zuckerberg
5,Warren Buffett,$96 B,United States,Berkshire Hathaway,6,90.0,Finance & Investments,https://www.forbes.com/profile/warren-buffett


**Now, let's start the web scraping process!First of all, we need to create a new column called Education_info to store the to be scraped content. We will implement the beautifulsoup package of python. We will loop through each billionaire's profile and locate the education tag. Consequently, we will avoid any nans and append the results in a temporal list**

In [9]:
#create a new column called Education_info, where we'll save the final results in
df['Education_info'] = df['urls']

Education_scraped = []
for url in df['urls']:
    try:
        r = requests.get(url)
        soup = BeautifulSoup(r.text)
    
        try:
             # locate to the "Education" tag of each person
            Education  = soup.find('span',text = 'Education')
            
            # go to the parent tag of Education, and access the content of the next tag of Education
            Education = Education.parent.contents[1].text
            
        except:
            Education = None
        
    except MissingSchema:
        Education = None
        
    #  save the scraped string in the temporal list
    Education_scraped.append(Education)

**Let's take a look at the data we have accquired from scraping**

In [10]:
Education_scraped[:10]

['Bachelor of Arts/Science, Princeton University',
 'Bachelor of Arts/Science, University of Pennsylvania',
 'Drop Out, Harvard University',
 'Drop Out, Harvard University',
 'Master of Science, Columbia University; Bachelor of Arts/Science, University of Nebraska Lincoln',
 'Drop Out, University of Chicago; Drop Out, University of Illinois, Urbana-Champaign',
 'Master of Science, Stanford University; Bachelor of Arts/Science, University of Michigan',
 'Master of Science, Stanford University; Bachelor of Arts/Science, University of Maryland, College Park',
 'Bachelor of Arts/Science, Harvard University; Drop Out, Stanford University',
 'Bachelor of Arts/Science, Trinity University']

**It is time to store these education information of billionaires into our dataframe's Education_info and take a look at our dataset**

In [11]:
df['Education_info'] = Education_scraped
df.head()

,Name,NetWorth,Country,Source,Rank,Age,Industry,urls,Education_info
0,Jeff Bezos,$177 B,United States,Amazon,1,57.0,Technology,https://www.forbes.com/profile/jeff-bezos,"Bachelor of Arts/Science, Princeton University"
1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,49.0,Automotive,https://www.forbes.com/profile/elon-musk,"Bachelor of Arts/Science, University of Pennsy..."
3,Bill Gates,$124 B,United States,Microsoft,4,65.0,Technology,https://www.forbes.com/profile/bill-gates,"Drop Out, Harvard University"
4,Mark Zuckerberg,$97 B,United States,Facebook,5,36.0,Technology,https://www.forbes.com/profile/mark-zuckerberg,"Drop Out, Harvard University"
5,Warren Buffett,$96 B,United States,Berkshire Hathaway,6,90.0,Finance & Investments,https://www.forbes.com/profile/warren-buffett,"Master of Science, Columbia University; Bachel..."


**Last but not least, we want to export the current data as a brand new csv file for the convenience of further cleaing and later analysis**

In [12]:
# save the df as a csv file
df.to_csv('billionaire_scraped.csv')

**Let's read in the billionaire_scraped csv file and assign it to df_colleege**

In [13]:
# read in billionaire dataset
df_college = pd.read_csv('billionaire_scraped.csv')
df_college

,Unnamed: 0,Name,NetWorth,Country,Source,Rank,Age,Industry,urls,Education_info
0,0,Jeff Bezos,$177 B,United States,Amazon,1,57.0,Technology,https://www.forbes.com/profile/jeff-bezos,"Bachelor of Arts/Science, Princeton University"
1,1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,49.0,Automotive,https://www.forbes.com/profile/elon-musk,"Bachelor of Arts/Science, University of Pennsy..."
2,3,Bill Gates,$124 B,United States,Microsoft,4,65.0,Technology,https://www.forbes.com/profile/bill-gates,"Drop Out, Harvard University"
3,4,Mark Zuckerberg,$97 B,United States,Facebook,5,36.0,Technology,https://www.forbes.com/profile/mark-zuckerberg,"Drop Out, Harvard University"
4,5,Warren Buffett,$96 B,United States,Berkshire Hathaway,6,90.0,Finance & Investments,https://www.forbes.com/profile/warren-buffett,"Master of Science, Columbia University; Bachel..."
...,...,...,...,...,...,...,...,...,...,...
719,2732,Vlad Tenev,$1 B,United States,stock trading,2674,34.0,Finance & Investments,https://www.forbes.com/profile/vlad-tenev,"Bachelor of Arts/Science, Stanford University"
720,2739,Vlad Vendrow & family,$1 B,United States,software,2674,53.0,Technology,https://www.forbes.com/profile/vlad-vendrow-&-...,NaN
721,2742,J. Wayne Weaver,$1 B,United States,Shoes,2674,85.0,Diversified,https://www.forbes.com/profile/j.-wayne-weaver,NaN
722,2743,Sandy Weill,$1 B,United States,Citigroup,2674,88.0,Finance & Investments,https://www.forbes.com/profile/sandy-weill,"Bachelor of Arts/Science, Cornell University"


**Let's check the number of nan values in the current dataset.**

In [14]:
df_college.isnull().sum()

Unnamed: 0          0
Name                0
NetWorth            0
Country             0
Source              0
Rank                0
Age                 4
Industry            0
urls                0
Education_info    181
dtype: int64

**We want to drop all the rows that contain nan values in the Education_info because our research question focuses on the correlation between drop out rates of univerisities and the US billionaires. If the billionaires' education info includes a nan value, then this billionaire essentially doesn't provide us any valuable information. In addition, we only want to include billionaires' undergraduate education information. Therefore, we filter those who only contain PHD or Master degrees out.**

In [15]:
df_college = df_college.dropna(subset=['Education_info'])
df_college = df_college[df_college['Education_info'].str.contains('Bachelor|Drop Out')].reset_index()
df_college

,index,Unnamed: 0,Name,NetWorth,Country,Source,Rank,Age,Industry,urls,Education_info
0,0,0,Jeff Bezos,$177 B,United States,Amazon,1,57.0,Technology,https://www.forbes.com/profile/jeff-bezos,"Bachelor of Arts/Science, Princeton University"
1,1,1,Elon Musk,$151 B,United States,"Tesla, SpaceX",2,49.0,Automotive,https://www.forbes.com/profile/elon-musk,"Bachelor of Arts/Science, University of Pennsy..."
2,2,3,Bill Gates,$124 B,United States,Microsoft,4,65.0,Technology,https://www.forbes.com/profile/bill-gates,"Drop Out, Harvard University"
3,3,4,Mark Zuckerberg,$97 B,United States,Facebook,5,36.0,Technology,https://www.forbes.com/profile/mark-zuckerberg,"Drop Out, Harvard University"
4,4,5,Warren Buffett,$96 B,United States,Berkshire Hathaway,6,90.0,Finance & Investments,https://www.forbes.com/profile/warren-buffett,"Master of Science, Columbia University; Bachel..."
...,...,...,...,...,...,...,...,...,...,...,...
495,710,2685,Larry Fink,$1 B,United States,money management,2674,68.0,Finance & Investments,https://www.forbes.com/profile/larry-fink,"Master of Business Administration, Anderson Sc..."
496,712,2701,David Katzman,$1 B,United States,teeth aligners,2674,61.0,Service,https://www.forbes.com/profile/david-katzman,"Bachelor of Arts/Science, Michigan State Unive..."
497,713,2702,Bom Kim,$1 B,United States,online retailing,2674,42.0,Technology,https://www.forbes.com/profile/bom-kim,"Bachelor of Arts/Science, Harvard University"
498,719,2732,Vlad Tenev,$1 B,United States,stock trading,2674,34.0,Finance & Investments,https://www.forbes.com/profile/vlad-tenev,"Bachelor of Arts/Science, Stanford University"


In [16]:
df_college.shape

(500, 11)

**Since the Unnamed:0 and urls columns are irrelevant information to our analysis, we will drop them from the existing dataset.**

In [17]:
df_college = df_college[['Name', 'NetWorth','Source','Rank','Age','Industry','Education_info']]
df_college

,Name,NetWorth,Source,Rank,Age,Industry,Education_info
0,Jeff Bezos,$177 B,Amazon,1,57.0,Technology,"Bachelor of Arts/Science, Princeton University"
1,Elon Musk,$151 B,"Tesla, SpaceX",2,49.0,Automotive,"Bachelor of Arts/Science, University of Pennsy..."
2,Bill Gates,$124 B,Microsoft,4,65.0,Technology,"Drop Out, Harvard University"
3,Mark Zuckerberg,$97 B,Facebook,5,36.0,Technology,"Drop Out, Harvard University"
4,Warren Buffett,$96 B,Berkshire Hathaway,6,90.0,Finance & Investments,"Master of Science, Columbia University; Bachel..."
...,...,...,...,...,...,...,...
495,Larry Fink,$1 B,money management,2674,68.0,Finance & Investments,"Master of Business Administration, Anderson Sc..."
496,David Katzman,$1 B,teeth aligners,2674,61.0,Service,"Bachelor of Arts/Science, Michigan State Unive..."
497,Bom Kim,$1 B,online retailing,2674,42.0,Technology,"Bachelor of Arts/Science, Harvard University"
498,Vlad Tenev,$1 B,stock trading,2674,34.0,Finance & Investments,"Bachelor of Arts/Science, Stanford University"


**The Education_info column is in an extremely haphazard format. Therefore, we want to construct a function to standardize the content into straightforward names of undergraduate institutions.**

In [18]:
def standardize_name(string):
    string = string.strip()
    string = string.replace('Bachelor of Arts/Science,','')
    string = string.replace('Master of Science, ','')
    string = string.replace('Master of Business Administration,','')
    string = string.replace('Drop Out,','')
    string = string.replace('Anderson School of Management;','')
    string = string.replace('Columbia University;','')
    string = string.replace('Doctor of Jurisprudence,','')
    string = string.replace('Bachelor of Science,','')
    string = string.replace('Diploma, The Hotchkiss School;','')
    string = string.replace('Harvard Business School;  ','')
    string = string.replace('Master of Arts/Science,','')
    string = string.replace('Stanford University;  ','')
    string = string.replace('Harvard University;  ','')
    string = string.replace('; LLB, Wayne State University','')
    string = string.replace('Massachusetts Institute of Technology; ','')
    string = string.replace('Doctorate, ','')
    string = string.replace('Princeton University; ','')
    string = string.replace('University of California, San Diego; ','')
    string = string.replace(', The Wharton School','')
    string = string.replace('New York University;  ','')
    string = string.replace('David A. Tepper School of Business;  ','')
    string = string.replace('Bachelor of Engineering, ','')
    string = string.replace(';  Samuel Curtis Johnson Graduate School of Management','')
    string = string.replace('Doctorate, California Institute of Technology;  ','')
    string = string.replace(';  Stanford Business School','')
    string = string.replace('Delhi University;  Delhi University; ','')
    string = string.replace('Stanford Graduate School of Business;  ','')
    string = string.replace('LLB, ','')
    string = string.replace('Cleveland State University;  ','')
    string = string.replace('; Law Degree, New York University','')
    string = string.replace('Cleveland State University;  ','')
    string = string.replace('University of Southern California; ','')
    string = string.replace('Cornell University; ','')
    string = string.replace('Georgetown University;  ','')
    string = string.replace('; Masters of Public Health, UCLA;  University of California, Irvine','')
    string = string.replace(';  Harvard Law School','')
    string = string.replace('Colby College;  ','')
    string = string.replace(';  University of Western Ontario','')
    string = string.replace('; Master of Philosophy, Oxford University','')
    string = string.replace('Bachelor of Arts/Economics, ','')
    string = string.replace('Columbia Business School;  ','')
    string = string.replace('INSEAD;  ','')
    string = string.replace('; Ph.D, University of Wisconsin Madison','')
    string = string.replace('; California Polytechnic State University','')
    string = string.replace('Medical Doctor, Hahnemann Medical College;  ','')
    string = string.replace('CW Post College; ','')
    string = string.replace('Lake Forest College;  ','')
    string = string.replace('; Law Degree, Northwestern University','')
    string = string.replace('Stanford Graduate School of Business;  ','')
    string = string.replace(';  University of Madras','')
    string = string.replace(';  University of Phoenix','')
    string = string.replace(' University of Pennsylvania;  ','')
    string = string.replace(', Stephen M. Ross School of Business','')
    string = string.replace('Indian Institute of Technology, Bombay;  ','')
    string = string.replace(';  Harvard Business School','')
    string = string.replace('Bachelor of Science in Engineering, ','')
    string = string.replace('Stanford Business School;  ','')
    string = string.replace('; Medical Doctor, Indiana University','')
    string = string.replace(';  University of Chicago;  University of Pennsylvania;  University of Pennsylvania','')
    string = string.replace('Columbia Business School;  ','')
    string = string.replace(';  The University of Chicago; Williams College','')
    string = string.replace('; Santa Clara University','')
    string = string.strip()
    string = string.replace(';  University of Illinois, Urbana-Champaign','')
    string = string.replace('; University of California, Berkeley','')
    string = string.replace(';  University of California, Berkeley','')
    string = string.replace('Bachelor of Technology, ','')
    string = string.replace(';  Tippie School of Management','')
    string = string.replace('Bachelor of Business Administration, ','')
    string = string.replace(';  Wake Forest University School of Business','')
    string = string.replace('; Master of Arts, University of Arkansas','')
    string = string.replace('Diploma, High School;  ','')
    string = string.replace('Law Degree, American University;  ','')
    string = string.replace('Master, Drexel University;  ','')
    string = string.replace('Dartmouth College;  ','')
    string = string.replace('Robert J. Trulaske, Sr. College of Business;  ','')
    string = string.replace('; Johns Hopkins University;  Yale University','')
    string = string.replace(';  UC Hastings','')
    string = string.replace(';  University of Witwatersrand; Medical Doctor, University of Witwatersrand','')
    string = string.replace(';  Columbia Business School','')
    string = string.replace('; Diploma, High School','')
    string = string.replace(';  University of Missouri','')
    string = string.replace('Master of Laws, University of Michigan;  ','')
    string = string.replace('Broad College of Business;  ','')
    string = string.replace(';  Rice University','')
    string = string.replace('; North Carolina State University; North Carolina State University','')
    string = string.replace('; Stanford University','')
    string = string.replace('; Master of Arts, Western Michigan University','')
    string = string.replace('; University of California, Los Angeles; University of California, Los Angeles','')
    string = string.replace('Dickinson College; ','')
    string = string.replace(';  University of Houston','')
    string = string.replace('MIT Sloan School of Management','Massachusetts Institute of Technology')
    string = string.replace('Christ Church, Oxford University;  ','')
    string = string.replace('California State Polytechnical University; ','')
    string = string.replace('; University of Minnesota','')
    string = string.replace(';  University of Michigan','')
    string = string.replace('Bowdoin College; ','')
    string = string.replace('; University of Illinois, Urbana-Champaign','')
    string = string.replace(';  Leonard N. Stern School of Business','')
    string = string.replace('Creighton University;  ','')
    string = string.replace(';  University of Houston','')
    string = string.replace('Claremont McKenna College;  ','')
    string = string.replace(';  University of Chicago','')
    string = string.replace('; Master of Science in Engineering, Stanford University ','')
    string = string.replace(';  Stanford University','')
    string = string.replace(';  University of Southern California','')
    string = string.replace('Leonard N. Stern School of Business;  ','')
    string = string.replace('; Tsinghua University','')
    string = string.replace('; Master of Architecture, University of California, Berkeley','')
    string = string.replace('Indian Institute of Technology;  ','')
    string = string.replace('; Master of Science in Engineering, Stanford University','')
    string = string.replace(';  The University of Texas at Austin','')
    string = string.replace(';  Southwestern Law School','')
    string = string.replace(';  University of Pennsylvania','')
    string = string.replace('; Vanderbilt University','')
    string = string.replace('New York Law School;  ','')
    string = string.replace('Chinese Culture University;  ','')
    string = string.replace('Associate in Arts/Science, ','')
    string = string.replace(';  Southern College of Optometry; Southern College of Optometry','')
    string = string.replace('Brooklyn Law School;  ','')
    string = string.replace(';  Washington University in St. Louis','')
    string = string.replace('Harvard Law School; ','')
    string = string.replace(';  Stanford Graduate School of Business','')
    string = string.replace('; Master of Arts, Northern Illinois University','')
    string = string.replace(';  Vanderbilt University','')
    string = string.replace(';  Clark University','')
    string = string.replace(';  Pomona College; Ph.D, University of California, San Diego','')
    string = string.replace('; University of New Orleans; Ph.D, University of Utah','')
    string = string.replace(';  Washington & Lee University','')
    string = string.replace(';  Tufts University','')
    string = string.replace('Long Island University; ','')
    string = string.replace(';  Southern Methodist University','')
    string = string.replace('; Medical Doctor, Yeshiva U Albert Einstein College of Medicine','')
    string = string.replace('; San Francisco State University','')
    string = string.replace('Ph.D, ','')
    string = string.replace(';  University of Virginia','')
    string = string.replace('City College of New York; ','')
    string = string.replace(';  University of Vermont','')
    string = string.replace('Law Degree, Pace University;  ','')
    string = string.replace('Tsinghua University; ','')
    string = string.replace('; The University of Texas at Austin','')
    string = string.replace(';  Harvard University','')
    string = string.replace('Robert H. Smith School of Business;  ','')
    string = string.replace('; Université de Paris VI; Ph.D, Université de Paris VI','')
    string = string.replace('; Université de Paris VI; Université de Paris VI','')
    string = string.replace(', The Netherlands; Erasmus University Rotterdam, The Netherlands','')
    string = string.replace('; Master of Arts, University of North Florida','')
    string = string.replace('; University of Kansas','')
    string = string.replace(';  University of California, Santa Barbara','')
    string = string.replace(';  New York Law School','')
    string = string.replace('Graduate School of Business','University')
    string = string.replace('; University of Pennsylvania','')
    string = string.replace(' Master of Arts, ','')
    string = string.replace('Medical Doctor, University of Virginia; ','')
    string = string.replace(' College of Law','')
    string = string.replace('; University of Pennsylvania','')
    string = string.strip()
    
    
    
    
    return string

**Let's apply the function we wrote just now and assign the new information into the Education column.**

In [19]:
df_college['Education_info'] = df_college['Education_info'].apply(standardize_name)
df_college = df_college.rename(columns = {'Education_info':'Education'})
df_college

/tmp/ipykernel_763/3908901607.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_college['Education_info'] = df_college['Education_info'].apply(standardize_name)


,Name,NetWorth,Source,Rank,Age,Industry,Education
0,Jeff Bezos,$177 B,Amazon,1,57.0,Technology,Princeton University
1,Elon Musk,$151 B,"Tesla, SpaceX",2,49.0,Automotive,University of Pennsylvania
2,Bill Gates,$124 B,Microsoft,4,65.0,Technology,Harvard University
3,Mark Zuckerberg,$97 B,Facebook,5,36.0,Technology,Harvard University
4,Warren Buffett,$96 B,Berkshire Hathaway,6,90.0,Finance & Investments,University of Nebraska Lincoln
...,...,...,...,...,...,...,...
495,Larry Fink,$1 B,money management,2674,68.0,Finance & Investments,"University of California, Los Angeles"
496,David Katzman,$1 B,teeth aligners,2674,61.0,Service,Michigan State University
497,Bom Kim,$1 B,online retailing,2674,42.0,Technology,Harvard University
498,Vlad Tenev,$1 B,stock trading,2674,34.0,Finance & Investments,Stanford University


**Let's take a look at the rank of the number of billionaire alumnis in each univeristy in descendning order.**

In [20]:
# group by college and rank by the number of billionaire alumni in descending order
grouped_df = df_college.groupby('Education').count().sort_values(by='Name', ascending=False)
grouped_df

,Name,NetWorth,Source,Rank,Age,Industry
Education,,,,,,
Stanford University,31,31,31,31,31,31
Harvard University,25,25,25,25,25,25
University of Pennsylvania,25,25,25,25,25,25
Yale University,13,13,13,13,13,13
"University of California, Berkeley",12,12,12,12,12,12
...,...,...,...,...,...,...
Ohio Northern University,1,1,1,1,1,1
Ohio State University,1,1,1,1,1,1
Oklahoma State University,1,1,1,1,1,1


**From the college instituion details dataset we found online(https://data.world/databeats/college-completion), we stored their names in a csv file named cc_institution_details.csv. Then, let's read in the dataset and assign it to cc_df**

In [21]:
# read in college instituion details dataset
cc_df = pd.read_csv('cc_institution_details.csv', encoding='Latin-1')
cc_df.head()

,unitid,chronname,city,state,level,control,basic,hbcu,flagship,long_x,...,vsa_grad_after6_transfer,vsa_grad_elsewhere_after6_transfer,vsa_enroll_after6_transfer,vsa_enroll_elsewhere_after6_transfer,similar,state_sector_ct,carnegie_ct,counted_pct,nicknames,cohort_size
0,100654,Alabama A&M University,Normal,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.568502,...,36.4,5.6,17.2,11.1,232937|100724|405997|113607|139533|144005|2285...,13,386,99.7|07,NaN,882.0
1,100663,University of Alabama at Birmingham,Birmingham,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.809170,...,NaN,NaN,NaN,NaN,196060|180461|201885|145600|209542|236939|1268...,13,106,56.0|07,UAB,1376.0
2,100690,Amridge University,Montgomery,Alabama,4-year,Private not-for-profit,Baccalaureate Colleges--Arts & Sciences,NaN,NaN,-86.174010,...,NaN,NaN,NaN,NaN,217925|441511|205124|247825|197647|221856|1353...,16,252,100.0|07,NaN,3.0
3,100706,University of Alabama at Huntsville,Huntsville,Alabama,4-year,Public,Research Universities--very high research acti...,NaN,NaN,-86.638420,...,0.0,0.0,0.0,0.0,232186|133881|196103|196413|207388|171128|1900...,13,106,43.1|07,UAH,759.0
4,100724,Alabama State University,Montgomery,Alabama,4-year,Public,Masters Colleges and Universities--larger prog...,X,NaN,-86.295677,...,NaN,NaN,NaN,NaN,100654|232937|242617|243197|144005|241739|2354...,13,386,88.0|07,ASU,1351.0


**Let's take a look at the dimension of the dataset.**

In [22]:
cc_df.shape

(3798, 62)

**We will only keep the columns that have information regarding names, states, controls of colleges, because these information are crucial for us to establish correlations between universities and billionaires. In addition, the grad_100_value and grad_150_value columns indicate the percentage of students who graduated in four years and five years.We can later calculate the dropout rate by subtracting them from 100%.**

In [23]:
# Filter out the columns we don't need
cc_df = cc_df[['chronname', 'state', 'control', 'grad_100_value', 'grad_150_value']]

**Let's take a look at the quantity of the nans.**

In [24]:
cc_df.isnull().sum()

chronname           0
state               0
control             0
grad_100_value    331
grad_150_value    331
dtype: int64

**We want to drop all the rows that contain nans in the grad_100_value and grad_150_value column because nans with such high quantity will for sure cause trouble in our analysis.**

In [25]:
# Drop any rows that contain missing data
cc_df = cc_df.dropna(subset=['grad_100_value', 'grad_150_value'])

**For the sake of clarity, we want to rename chronname and control to name and type**

In [26]:
cc_df = cc_df.rename(columns={"chronname": "name", "control": "type"})

**Since we are interested in the the dropout rates of these universities, we subtract the values in grad_100_value and grad_150_value from 100 to obtain the dropout rates. Then, we assign these values to new columns of nongrad_100_value and nongrad_150_value.**

In [27]:
# Calculate the nongraduate rate columns
cc_df['nongrad_100_value'] = 100 - cc_df['grad_100_value']
cc_df['nongrad_150_value'] = 100 - cc_df['grad_150_value']
cc_df

,name,state,type,grad_100_value,grad_150_value,nongrad_100_value,nongrad_150_value
0,Alabama A&M University,Alabama,Public,10.0,29.1,90.0,70.9
1,University of Alabama at Birmingham,Alabama,Public,29.4,53.5,70.6,46.5
2,Amridge University,Alabama,Private not-for-profit,0.0,66.7,100.0,33.3
3,University of Alabama at Huntsville,Alabama,Public,16.5,48.4,83.5,51.6
4,Alabama State University,Alabama,Public,8.8,25.2,91.2,74.8
...,...,...,...,...,...,...,...
3793,Grace College of Divinity,North Carolina,Private not-for-profit,0.0,0.0,100.0,100.0
3794,John Paul the Great Catholic University,California,Private not-for-profit,51.5,51.5,48.5,48.5
3795,Chamberlain College of Nursing-Missouri,Missouri,Private for-profit,37.5,37.5,62.5,62.5
3796,Minneapolis Media Institute,Minnesota,Private for-profit,42.1,55.9,57.9,44.1
